In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2 
import skimage

from sklearn.model_selection import train_test_split

## Metrics
from sklearn.metrics import accuracy_score

## tensorflow & Keras
import tensorflow as tf   ## i will use tf for every thing and for keras using tf.keras

np.random.seed(42) # My lucky number

model_path = 'final_model.h5'

In [ ]:
TRAIN_DIR = './train/'
TEST_DIR = './test/'

train_frames = os.listdir(TRAIN_DIR)
train_frames.sort()

test_frames = os.listdir(TEST_DIR)
test_frames.sort()

train_labels = pd.read_csv('train.csv')
test_labels = pd.read_csv('test.csv')

In [ ]:
train_labels['label'].unique().size

In [ ]:
len(train_frames), len(test_frames)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# parameters for your graph
n_rows = 8
n_cols = 8

pics_index = 0

In [ ]:
# set the matplotlib figure
fig = plt.gcf()
fig.set_size_inches(4*n_cols, 4*n_rows)

pics_index += 32

next_pix = [os.path.join(TRAIN_DIR, fname)
            for fname in train_frames[pics_index-32 : pics_index]
           ]

for index, img_path in enumerate(next_pix):
    sp = plt.subplot(n_cols, n_rows, index+1)
    sp.axis('Off') # Don't show axes (or gridlines)
    plt.title('label = ' + str(train_labels.iloc[index]['label']), fontsize=28)
    img = mpimg.imread(img_path)
    plt.imshow(img)
plt.show()

In [ ]:
img = mpimg.imread(TRAIN_DIR + train_frames[2])
img.shape

In [ ]:
full_train_set = np.empty((13440, 32, 32, 1), dtype=np.float32)
full_test_set = np.empty((3360, 32, 32, 1), dtype=np.float32)

for index, img_path in enumerate(train_frames):
    img = mpimg.imread(os.path.join(TRAIN_DIR, img_path))
    img_gray = skimage.color.rgb2gray(img[:,:,:3])
    th, image = cv2.threshold(img_gray*255, 127, 255, cv2.THRESH_BINARY)
    full_train_set[index] = image.reshape(32, 32, 1)

for index, img_path in enumerate(test_frames):
    img = mpimg.imread(os.path.join(TEST_DIR, img_path))
    img_gray = skimage.color.rgb2gray(img[:,:,:3])
    th, image = cv2.threshold(img_gray*255, 127, 255, cv2.THRESH_BINARY)
    full_test_set[index] = image.reshape(32, 32, 1)

In [ ]:
X_train = full_train_set
y_train = train_labels['label'].values

X_test = full_test_set

In [ ]:
# input_shape = (32, 32, 3)
# num_classes = 29

# def model_cnn(input_shape=input_shape, num_classes=num_classes):   
#     model = tf.keras.models.Sequential()

#     # Add convolutional layer consisting of 32 filters and shape of 3x3 with ReLU activation
#     # We want to preserve more information for following layers so we use padding
#     # 'Same' padding tries to pad evenly left and right, 
#     # but if the amount of columns to be added is odd, it will add the extra column to the right
#     model.add(tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape = input_shape))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Conv2D(32, kernel_size = (3,3), activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())

#     # Add convolutional layer consisting of 32 filters and shape of 5x5 with ReLU activation
#     # We give strides=2 for space between each sample on the pixel grid
#     model.add(tf.keras.layers.Conv2D(32, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     # Dropping %40 of neurons
#     model.add(tf.keras.layers.Dropout(0.4))
    
#     model.add(tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Conv2D(64, kernel_size = (3,3), activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Conv2D(64, kernel_size = (5,5), strides=2, padding='same', activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.Dropout(0.4))

#     model.add(tf.keras.layers.Conv2D(128, kernel_size = 4, activation='relu'))
#     model.add(tf.keras.layers.BatchNormalization())
#     # To be able to merge into fully connected layer we have to flatten
#     model.add(tf.keras.layers.Flatten())
#     model.add(tf.keras.layers.Dropout(0.4))
#     # Lets add softmax activated neurons as much as number of classes
#     model.add(tf.keras.layers.Dense(num_classes, activation = "softmax"))
#     # Compile the model with loss and metrics
#     model.compile(optimizer =  tf.keras.optimizers.Adam() , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])
    
#     return model

# model = model_cnn()

In [ ]:
droprate = 0.6
num_classes = 29
size = 32

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(256, kernel_size=(2, 2), strides=(1, 1), activation='elu', input_shape=(size, size, 1)))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(256, kernel_size=(4, 4), strides=(1, 1), activation='elu', padding='valid'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(256, kernel_size=(4, 4), strides=(1, 1), activation='elu', padding='valid'))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Conv2D(256, kernel_size=(4, 4), strides=(1, 1), activation='elu', padding='valid'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(droprate))
model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(512, activation='elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(droprate))

model.add(tf.keras.layers.Dense(256, activation='elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(droprate))

model.add(tf.keras.layers.Dense(128, activation='elu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(droprate))

model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer =  tf.keras.optimizers.Adam() , loss = "sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_acc', min_delta=0.01, patience=4, verbose=1)
cb = [es]

In [ ]:
history = model.fit(X_train, y_train, validation_split=0.2, callbacks=cb,
                    epochs=85, batch_size=32)

In [ ]:
# model.save(model_path)

In [ ]:
# model = tf.keras.models.load_model(model_path)

In [ ]:
pd.DataFrame(history.history).plot(figsize=(10, 6))

In [ ]:
y_preds_classes = np.argmax(model.predict(X_test), axis=-1)

In [ ]:
test_labels['label'] = y_preds_classes

In [ ]:
test_labels

In [ ]:
test_labels[['id', 'label']].to_csv('/kaggle/working/submission.csv', index=False)